## TaskOperation API

@Author: 吴炜坤

@email：weikun.wu@xtalpi.com/weikunwu@163.com

本章节主要详尽地介绍TaskOperation的API，主要参考页面:
> http://new.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/TaskOperations/TaskOperations-RosettaScripts#specialized-operations_packer-behavior-modification

TaskOperation从构建逻辑上来分类共计有两种类型:

* Residue Level TaskOperations: 根据Selector设定选择范围内位点的Rotamer自由度(手动挡);
* Specialized Operations: 根据预设好逻辑，对位点进行全局Rotamer操作(自动挡);

## 一、Residue Level TaskOperations

Residue Level TaskOperations(RLT)一般需要配合Selector来指定操作的范围。用户可以直观地将RLT理解为一个自定义版本的Specialized Operations。特别注意的是:RLT是无法直接被TaskFactory所读取，其必须通过OperateOnResidueSubset函数来生成一个标准的TaskOperations。

Rosetta中目前支持的所有RTL的列表:
- RestrictToRepackingRLT
- PreventRepackingRLT
- RestrictAbsentCanonicalAASExceptNativeRLT
- RestrictAbsentCanonicalAASRLT
- DisallowIfNonnativeRLT
- IncludeCurrentRLT
- ExtraRotamersGenericRLT

In [6]:
# 初始化PyRosetta并读取一段螺旋结构的PDB。
from pyrosetta import *
init()
pose = pose_from_pdb('./data/helix.pdb')

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release r274 2020.50+release.1295438cd4b 1295438cd4bd2be39c9dbbfab8db669ab62415ab http://www.pyrosetta.org 2020-12-12T00:30:01
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.6/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-266372846 seed_offset=0 real_seed=-266372846 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-266372846 RG_type=mt19937
core.import_pose.import_p

**RestrictToRepackingRLT在上一章节中已经介绍，此处不再赘述。**

In [17]:
# 预先定义好 氨基酸的作用范围。
from pyrosetta.rosetta.core.pack.task.operation import *
from pyrosetta.rosetta.core.pack.task.operation import OperateOnResidueSubset
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
from pyrosetta.rosetta.core.pack.task import TaskFactory
# 选择氨基酸范围
select_pos = ResidueIndexSelector('2,3,4,5,6,7,8,9')

#### 1.1 PreventRepackingRLT

将选择区域的氨基酸Rotamer自由度完全关闭，其侧链构象维持不变。

In [18]:
# 使用OperateOnResidueSubset生成TaskOperations
packing_taskop = OperateOnResidueSubset(PreventRepackingRLT(), select_pos, False)

# 将TaskOperations加载至TaskFactory中
pack_tf = TaskFactory()
pack_tf.push_back(packing_taskop)

# 生成PackerTask
packer_task = pack_tf.create_task_and_apply_taskoperations(pose)
print(packer_task)

#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	TRUE	ALA:NtermProteinFull,CYS:NtermProteinFull,ASP:NtermProteinFull,GLU:NtermProteinFull,PHE:NtermProteinFull,GLY:NtermProteinFull,HIS:NtermProteinFull,HIS_D:NtermProteinFull,ILE:NtermProteinFull,LYS:NtermProteinFull,LEU:NtermProteinFull,MET:NtermProteinFull,ASN:NtermProteinFull,PRO:NtermProteinFull,GLN:NtermProteinFull,ARG:NtermProteinFull,SER:NtermProteinFull,THR:NtermProteinFull,VAL:NtermProteinFull,TRP:NtermProteinFull,TYR:NtermProteinFull
2	FALSE	FALSE	
3	FALSE	FALSE	
4	FALSE	FALSE	
5	FALSE	FALSE	
6	FALSE	FALSE	
7	FALSE	FALSE	
8	FALSE	FALSE	
9	FALSE	FALSE	
10	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
11	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
12	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
13	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,

#### 1.2 RestrictAbsentCanonicalAASExceptNativeRLT

将Rotamer自由度限定在给定的氨基酸类型列表，并允许保留当前位点氨基酸类型的Rotamer保留。

In [23]:
# 定义自由度
design_with_wt = RestrictAbsentCanonicalAASExceptNativeRLT()
design_with_wt.aas_to_keep('QKI')

# 使用OperateOnResidueSubset生成TaskOperations
packing_taskop = OperateOnResidueSubset(design_with_wt, select_pos, False)

# 将TaskOperations加载至TaskFactory中
pack_tf = TaskFactory()
pack_tf.push_back(packing_taskop)

# 生成PackerTask
packer_task = pack_tf.create_task_and_apply_taskoperations(pose)
print(packer_task)

#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	TRUE	ALA:NtermProteinFull,CYS:NtermProteinFull,ASP:NtermProteinFull,GLU:NtermProteinFull,PHE:NtermProteinFull,GLY:NtermProteinFull,HIS:NtermProteinFull,HIS_D:NtermProteinFull,ILE:NtermProteinFull,LYS:NtermProteinFull,LEU:NtermProteinFull,MET:NtermProteinFull,ASN:NtermProteinFull,PRO:NtermProteinFull,GLN:NtermProteinFull,ARG:NtermProteinFull,SER:NtermProteinFull,THR:NtermProteinFull,VAL:NtermProteinFull,TRP:NtermProteinFull,TYR:NtermProteinFull
2	TRUE	TRUE	GLU,ILE,LYS,GLN
3	TRUE	TRUE	ILE,LYS,LEU,GLN
4	TRUE	TRUE	ILE,LYS,GLN
5	TRUE	TRUE	ILE,LYS,GLN
6	TRUE	TRUE	ILE,LYS,GLN,TRP
7	TRUE	TRUE	ILE,LYS,GLN,VAL
8	TRUE	TRUE	GLU,ILE,LYS,GLN
9	TRUE	TRUE	ILE,LYS,GLN
10	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
11	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
12	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS

#### 1.3 RestrictAbsentCanonicalAASRLT

功能与RestrictAbsentCanonicalAASExceptNativeRLT类似，将Rotamer自由度限定在给定的氨基酸类型列表。唯一的差别在于RestrictAbsentCanonicalAASRLT会忘记当前的氨基酸类型。

In [25]:
# 定义自由度
design_to = RestrictAbsentCanonicalAASRLT()
design_to.aas_to_keep('QKI')

# 使用OperateOnResidueSubset生成TaskOperations
packing_taskop = OperateOnResidueSubset(design_to, select_pos, False)

# 将TaskOperations加载至TaskFactory中
pack_tf = TaskFactory()
pack_tf.push_back(packing_taskop)

# 生成PackerTask
packer_task = pack_tf.create_task_and_apply_taskoperations(pose)
print(packer_task)

#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	TRUE	ALA:NtermProteinFull,CYS:NtermProteinFull,ASP:NtermProteinFull,GLU:NtermProteinFull,PHE:NtermProteinFull,GLY:NtermProteinFull,HIS:NtermProteinFull,HIS_D:NtermProteinFull,ILE:NtermProteinFull,LYS:NtermProteinFull,LEU:NtermProteinFull,MET:NtermProteinFull,ASN:NtermProteinFull,PRO:NtermProteinFull,GLN:NtermProteinFull,ARG:NtermProteinFull,SER:NtermProteinFull,THR:NtermProteinFull,VAL:NtermProteinFull,TRP:NtermProteinFull,TYR:NtermProteinFull
2	TRUE	TRUE	ILE,LYS,GLN
3	TRUE	TRUE	ILE,LYS,GLN
4	TRUE	TRUE	ILE,LYS,GLN
5	TRUE	TRUE	ILE,LYS,GLN
6	TRUE	TRUE	ILE,LYS,GLN
7	TRUE	TRUE	ILE,LYS,GLN
8	TRUE	TRUE	ILE,LYS,GLN
9	TRUE	TRUE	ILE,LYS,GLN
10	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
11	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
12	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN

#### 1.4 DisallowIfNonnativeRLT

不允许突变为列表中指定氨基酸类型，但允许保留当前位点氨基酸类型的Rotamer保留。

In [47]:
# 定义自由度
not_design_to = DisallowIfNonnativeRLT()
not_design_to.disallow_aas('HLQKILRA')

# 使用OperateOnResidueSubset生成TaskOperations
packing_taskop = OperateOnResidueSubset(not_design_to, select_pos, False)

# 将TaskOperations加载至TaskFactory中
pack_tf = TaskFactory()
pack_tf.push_back(packing_taskop)

# 生成PackerTask
packer_task = pack_tf.create_task_and_apply_taskoperations(pose)
print(packer_task)

#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	TRUE	ALA:NtermProteinFull,CYS:NtermProteinFull,ASP:NtermProteinFull,GLU:NtermProteinFull,PHE:NtermProteinFull,GLY:NtermProteinFull,HIS:NtermProteinFull,HIS_D:NtermProteinFull,ILE:NtermProteinFull,LYS:NtermProteinFull,LEU:NtermProteinFull,MET:NtermProteinFull,ASN:NtermProteinFull,PRO:NtermProteinFull,GLN:NtermProteinFull,ARG:NtermProteinFull,SER:NtermProteinFull,THR:NtermProteinFull,VAL:NtermProteinFull,TRP:NtermProteinFull,TYR:NtermProteinFull
2	TRUE	TRUE	CYS,ASP,GLU,PHE,GLY,MET,ASN,PRO,SER,THR,VAL,TRP,TYR
3	TRUE	TRUE	CYS,ASP,GLU,PHE,GLY,LEU,MET,ASN,PRO,SER,THR,VAL,TRP,TYR
4	TRUE	TRUE	CYS,ASP,GLU,PHE,GLY,MET,ASN,PRO,GLN,SER,THR,VAL,TRP,TYR
5	TRUE	TRUE	CYS,ASP,GLU,PHE,GLY,LYS,MET,ASN,PRO,SER,THR,VAL,TRP,TYR
6	TRUE	TRUE	CYS,ASP,GLU,PHE,GLY,MET,ASN,PRO,SER,THR,VAL,TRP,TYR
7	TRUE	TRUE	CYS,ASP,GLU,PHE,GLY,MET,ASN,PRO,SER,THR,VAL,TRP,TYR
8	TRUE	TRUE	CYS,ASP,GLU,PHE,GLY,MET,ASN,PRO,SER,THR,VAL,TRP,TYR
9	TR

#### 1.5 IncludeCurrentRLT
设定Packer在执行期间，考虑Pose输入时的Rotamer状态。

In [57]:
# 使用OperateOnResidueSubset生成TaskOperations
packing_taskop = OperateOnResidueSubset(IncludeCurrentRLT(), select_pos, False)

# 将TaskOperations加载至TaskFactory中
pack_tf = TaskFactory()
pack_tf.push_back(packing_taskop)

# 生成PackerTask
packer_task = pack_tf.create_task_and_apply_taskoperations(pose)

# 检查是否Pack中是否包含input的rotamer构象:
packer_task.include_current(2)

True

In [58]:
# 检查是否Pack中是否包含input的rotamer构象:
packer_task.include_current(1)

False

#### 1.6 ExtraRotamersGenericRLT
设定在Rotamer采样中，是否考虑增加额外的数量，否则仅会考虑理想型的Rotamer。

## 二、Specialized Operations

### 2.1 Position/Identity Specification

### 2.2 Rotamer Specification

### 2.3 Packer Behavior Modification

### 2.4 Antibody and CDR Specific Operations